<a href="https://colab.research.google.com/github/albertchristian92/BBC-Text-Categorization/blob/main/Dicoding_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Pertama-tama set runtime setting to GPU pada google colabs agar dapat di run menggunakan GPU, sebab jika training menggunakan CPU akan consume time
#Runtime-> Change Runtime type
#Cek apakah sudah menggunakan GPU atau belum

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
#Cek GPU apa yang digunakan
!nvidia-smi

Thu Jul 15 11:15:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    30W /  70W |    222MiB / 15109MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#Import Library yang dibutuhkan
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Flatten, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Embedding

import matplotlib.pyplot as plt

#This code was tested with TensorFlow v2.4.1
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 2.5.0


Upload dataset ke google colabs:
Dataset yang saya gunakan:  https://www.kaggle.com/yufengdev/bbc-fulltext-and-category

In [4]:
#baca data dan tampilkan 5 sampel teratas pada dataset
df = pd.read_csv("bbc-text.csv")
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [5]:
# one-hot-encoding 
kategori = pd.get_dummies(df.category)
df_baru = pd.concat([df, kategori], axis=1)
df_baru = df_baru.drop(columns='category')
df_baru

,text,business,entertainment,politics,sport,tech
0,tv future in the hands of viewers with home th...,0,0,0,0,1
1,worldcom boss left books alone former worldc...,1,0,0,0,0
2,tigers wary of farrell gamble leicester say ...,0,0,0,1,0
3,yeading face newcastle in fa cup premiership s...,0,0,0,1,0
4,ocean s twelve raids box office ocean s twelve...,0,1,0,0,0
...,...,...,...,...,...,...
2220,cars pull down us retail figures us retail sal...,1,0,0,0,0
2221,kilroy unveils immigration policy ex-chatshow ...,0,0,1,0,0
2222,rem announce new glasgow concert us band rem h...,0,1,0,0,0
2223,how political squabbles snowball it s become c...,0,0,1,0,0


In [6]:
#ubah nilai-nilai dari dataframe ke dalam tipe data numpy array 
news_text = df_baru['text'].values
label = df_baru[['business', 'entertainment', 'politics', 'sport', 'tech']].values


In [7]:
#bagi dataset menjadi train dan test set
train_text,test_text,train_label,test_label=train_test_split(news_text,label,test_size=0.2)

In [16]:
#ubah setiap kata pada dataset kita ke dalam bilangan numerik dengan fungsi Tokenizer dan konversi setiap sampel menjadi sequences
max_kata = 1000
tokenizer = Tokenizer(num_words=max_kata, oov_token='x')

tokenizer.fit_on_texts(train_text) 
tokenizer.fit_on_texts(test_text)
 
sekuens_train = tokenizer.texts_to_sequences(train_text)
sekuens_test = tokenizer.texts_to_sequences(test_text)
 
padded_train = pad_sequences(sekuens_train) 
padded_test = pad_sequences(sekuens_test)

panjang_index_kata = len(tokenizer.word_index)


In [22]:
#set batch size, epoch, and drop ratio
batch_size = 16
num_epochs = 10
drop_ratio =0.5
embedding_dim = 300
n_class=5
max_len = 128


In [27]:
#buat model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=embedding_dim))
model.add(Bidirectional(LSTM(embedding_dim, kernel_initializer='orthogonal')))
model.add(Dense(embedding_dim, activation='relu', input_shape=(max_kata,)))
model.add(Dropout(drop_ratio))
model.add(Dense(n_class, activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 300)         1500000   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 600)               1442400   
_________________________________________________________________
dense_6 (Dense)              (None, 300)               180300    
_________________________________________________________________
dropout_4 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 1505      
Total params: 3,124,205
Trainable params: 3,124,205
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [29]:
history = model.fit(padded_train, train_label, epochs=num_epochs, batch_size=batch_size,
                    validation_data=(padded_test, test_label), verbose=2, validation_split=0.1)

Epoch 1/20
101/101 - 59s - loss: 1.6061 - accuracy: 0.2615 - val_loss: 1.4308 - val_accuracy: 0.4157
Epoch 2/20
101/101 - 56s - loss: 1.1053 - accuracy: 0.5574 - val_loss: 0.9969 - val_accuracy: 0.6348
Epoch 3/20
101/101 - 56s - loss: 0.6334 - accuracy: 0.7672 - val_loss: 0.9226 - val_accuracy: 0.6854
Epoch 4/20
101/101 - 56s - loss: 0.4979 - accuracy: 0.8433 - val_loss: 0.8355 - val_accuracy: 0.7303
Epoch 5/20
101/101 - 56s - loss: 0.4988 - accuracy: 0.8365 - val_loss: 0.7936 - val_accuracy: 0.7416
Epoch 6/20
101/101 - 57s - loss: 0.2952 - accuracy: 0.8908 - val_loss: 0.9382 - val_accuracy: 0.7247
Epoch 7/20
101/101 - 56s - loss: 0.2111 - accuracy: 0.9251 - val_loss: 0.8638 - val_accuracy: 0.8090
Epoch 8/20
101/101 - 56s - loss: 0.1262 - accuracy: 0.9588 - val_loss: 0.9602 - val_accuracy: 0.7978
Epoch 9/20
101/101 - 56s - loss: 0.0833 - accuracy: 0.9763 - val_loss: 1.0168 - val_accuracy: 0.8146
Epoch 10/20
101/101 - 56s - loss: 0.0763 - accuracy: 0.9775 - val_loss: 1.3426 - val_accura

In [30]:
#buat function untuk grafik
def show_final_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history["accuracy"], label="Train accuracy")
    ax[1].plot(history.epoch, history.history["val_accuracy"], label="Validation accuracy")
    ax[0].legend()
    ax[1].legend()

In [ ]:
#Cetak grafik dan nilai train dan test

show_final_history(history)
train_score=model.evaluate(padded_train,train_label,verbose=0)
test_score = model.evaluate(padded_test,test_label, verbose=0)
print("Nilai Train loss :",train_score[0])
print("Nilai Akurasi train :",train_score[1])
print("Nilai Test Loss:",test_score[0])
print("Nilai Akurasi Test :",test_score[1])

#save model   
model.save("model.h5")
print("Weights Saved")